---
#1. Preparation
##1.1 Download Stable-Diffusion Checkpoint

In [1]:
%%shell
mkdir -p pretrained/
cd pretrained/
wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt
cd ..

--2023-07-14 12:58:01--  https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt
Resolving huggingface.co (huggingface.co)... 13.35.166.36, 13.35.166.50, 13.35.166.114, ...
Connecting to huggingface.co (huggingface.co)|13.35.166.36|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/4c/37/4c372b4ebb57bbd02e68413d4951aa326d4b3cfb6e62db989e529c6d4b26fb21/14749efc0ae8ef0329391ad4436feb781b402f4fece4883c7ad8d10556d8a36a?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sd-v1-4-full-ema.ckpt%3B+filename%3D%22sd-v1-4-full-ema.ckpt%22%3B&Expires=1689598681&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4OTU5ODY4MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80Yy8zNy80YzM3MmI0ZWJiNTdiYmQwMmU2ODQxM2Q0OTUxYWEzMjZkNGIzY2ZiNmU2MmRiOTg5ZTUyOWM2ZDRiMjZmYjIxLzE0NzQ5ZWZjMGFlOGVmMDMyOTM5MWFkNDQzNmZlYjc4MWI0MDJmNGZlY2U

##1.2 Clone CelebBasis Code from [GitHub](https://github.com/ygtxr1997/CelebBasis/tree/main)

In [2]:
!git clone https://github.com/ygtxr1997/CelebBasis.git

Cloning into 'CelebBasis'...
remote: Enumerating objects: 764, done.
remote: Counting objects: 100% (764/764), done.
remote: Compressing objects: 100% (620/620), done.
remote: Total 764 (delta 113), reused 723 (delta 90), pack-reused 0
Receiving objects: 100% (764/764), 16.79 MiB | 11.90 MiB/s, done.
Resolving deltas: 100% (113/113), done.


##1.3 Copy StyleGAN Datasets and Third-Party Pretrained Weights from Your Own Google Drive
You should first copy [CelebBasis Google Drive](https://drive.google.com/drive/folders/181pJ3Y2Ffk0R8jVcErPT611aWQLCz7D9?usp=sharing) to your own [Google Drive](https://drive.google.com/drive/my-drive).
The Google Drive directory tree will be like:
```shell
My Drive/
 |--CelebBasis/
  |--datasets/
  |--pretrained/
```
Then, the following cell mounts your own Google Drive to this Colab Notebook and copies the datasets and weights files from the mounted folder.

In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/CelebBasis/datasets/ ./
!cp -r /content/drive/MyDrive/CelebBasis/pretrained/* ./pretrained/
drive.flush_and_unmount()
!cd ./datasets && tar -xf stylegan.tar && rm stylegan.tar
!cp ./pretrained/PIPNet/* ./CelebBasis/evaluation/face_align/PIPNet/weights
!mkdir -p ./CelebBasis/weights/
!cp -r ./pretrained/glint360k_cosface_r100_fp16_0.1/ ./CelebBasis/weights/

Mounted at /content/drive


## 1.4 Use PIP to Install the Python Dependencies and Uninstall the Needless Modules (which are preinstalled by Google Colab but not used)

In [4]:
!mkdir -p pip_installed/ && cd pip_installed/ && pip install -r ../CelebBasis/requirements.txt
!pip uninstall -r ./CelebBasis/requirements_uninstall.txt -y
!pip list |grep torch

Obtaining clip from git+https://github.com/openai/CLIP.git@a1d071733d7111c9c014f024669f959182114e33#egg=clip (from -r ../CelebBasis/requirements.txt (line 18))
  Cloning https://github.com/openai/CLIP.git (to revision a1d071733d7111c9c014f024669f959182114e33) to ./src/clip
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /content/pip_installed/src/clip
  Running command git rev-parse -q --verify 'sha^a1d071733d7111c9c014f024669f959182114e33'
  Running command git fetch -q https://github.com/openai/CLIP.git a1d071733d7111c9c014f024669f959182114e33
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
Obtaining taming_transformers from git+https://github.com/CompVis/taming-transformers.git@3ba01b241669f5ade541ce990f7650a3b8f65318#egg=taming_transformers (from -r ../CelebBasis/requirements.txt (line 141))
  Cloning https://github.com/CompVis/taming-transformers.git 

---
#2. Crop and Align Faces

In [5]:
!cd CelebBasis && chmod +x *.sh && bash ./00_align_face.sh ../datasets/stylegan/stylegan3-r-ffhq-1024x1024 ../datasets/stylegan/ffhq

/content/CelebBasis/evaluation/face_align/PIPNet/FaceBoxesV2/utils/build.py:13: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.core import setup
running build_ext
cythoning nms/cpu_nms.pyx to nms/cpu_nms.c
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/CelebBasis/evaluation/face_align/PIPNet/FaceBoxesV2/utils/nms/cpu_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'nms.cpu_nms' extension
creating build
creating build/temp.linux-x86_64-3.10
creating build/temp.linux-x86_64-3.10/nms
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror

#3. Train
* Limited by Google Colab GPU Memory ($<$16GB), we have to modify the `configs/stable-diffusion/aigc_id_colab.yaml` as follows:
```yaml
model:
  params:
    personalization_config:
      params:
        momentum: 0.9  # default: 0.99
data:
  params:
    batch_size: 1  # default: 2
    num_workers: 2  # default: 8
lightning:
  modelcheckpoint:
    params:
      every_n_train_steps: 400  # default: 200
  callbacks:
    image_logger:
      params:
        batch_frequency: 1200  # default: 600
  trainer:
    max_steps: 800  # default: 1600
```
* These modifications may increase the training time and downgrade the performance slightly.

In [ ]:
!cd CelebBasis && bash ./01_start_train.sh ../pretrained/sd-v1-4-full-ema.ckpt ./configs/stable-diffusion/aigc_id_colab.yaml

Global seed set to 23
Running on GPUs 0,
Loading model from ../pretrained/sd-v1-4-full-ema.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
Downloading: "https://github.com/DagnyT/hardnet/raw/master/pretrained/train_liberty_with_aug/checkpoint_liberty_with_aug.pth" to /root/.cache/torch/hub/checkpoints/checkpoint_liberty_with_aug.pth
100% 5.10M/5.10M [00:00<00:00, 60.8MB/s]
Downloading: 100% 939k/939k [00:00<00:00, 1.41MB/s]
Downloading: 100% 512k/512k [00:00<00:00, 1.01MB/s]
Downloading: 100% 389/389 [00:00<00:00, 229kB/s]
Downloading: 100% 905/905 [00:00<00:00, 784kB/s]
Downloading: 100% 4.41k/4.41k [00:00<00:00, 3.39MB/s]
Downloading: 100% 1.59G/1.59G [00:23<00:00, 74.0MB/s]
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializin

#4. Test
Modify the following `"traininYYYY-MM-DDTHH-MM-SS_celebbasis/"` to the project folder under `./CelebBasis/logs/` and run the test.

The generated images will be output under `./CelebBasis/outputs/traininYYYY-MM-DDTHH-MM-SS_celebbasis/`. Enjoy it!

In [13]:
!cd CelebBasis && bash ./02_start_test.sh "../pretrained/sd-v1-4-full-ema.ckpt" "./infer_images/example_prompt.txt" "training2023-07-13T14-55-56_celebbasis" 4

logs/training2023-07-13T14-55-56_celebbasis/configs/training2023-07-13T14-55-56-project.yaml
embeddings_gs-1599.pt
Global seed set to 42
Loading model from ../pretrained/sd-v1-4-full-ema.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.post_layernorm.bias', 'vision_model.encoder.layers.1.layer_norm1.bias', 'vision_model.encoder.layers.22.layer_norm2.bias', 'vision_model.encoder.layers.5.layer_norm1.weight', 'vision_model.encoder.layers.8.self_attn.out_proj.weight', 'vision_model.encoder.layers.10.self_attn.k_proj.bias', 'vision_model.encoder.layers.9.self_attn.out_proj.weight', 'vision_model.encoder.layers.10.self_attn.k_proj.w

In [1]:
!nvidia-smi

Thu Jul 13 15:24:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------